In [ ]:
# Exercise sheet 12 with the files train.csv and test.csv. Load the data set into your console.

# Task 1
# In Moodle, you can find the files train.csv and test.csv. Load the data set into your console. The data sets contain questions and their associated coarse- and fine-grained labels from the TREC data set. 

# The dataset consists of 6 coarse-grained categories and 50 fine-grained cate- gories. 
# In the coming tasks, we will compare different the classification rates of different models for this multi-label classification task. Preprocess the data so that it is suitable for the different pipelines in the upcoming tasks.


In [1]:
import pandas as pd

# Load the datasets
file_path = '/Users/oayanwale/Downloads/NLP_Exercise_24_25/Data'
train_df = pd.read_csv(f"{file_path}/train.csv")
test_df = pd.read_csv(f"{file_path}/test.csv")

# Show the first few rows to understand the structure of the data
print(train_df.head())
print(test_df.head())

   label-coarse  label-fine                                               text
0             0           0  How did serfdom develop in and then leave Russ...
1             1           1   What films featured the character Popeye Doyle ?
2             0           0  How can I find a list of celebrities ' real na...
3             1           2  What fowl grabs the spotlight after the Chines...
4             2           3                    What is the full form of .com ?
   label-coarse  label-fine                                      text
0             4          40      How far is it from Denver to Aspen ?
1             5          21  What county is Modesto , California in ?
2             3          12                         Who was Galileo ?
3             0           7                         What is an atom ?
4             4           8          When did Hawaii become a state ?


In [2]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Initialize necessary components for preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [token for token in tokens if not token.isdigit()]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens).strip()

# Apply preprocessing to the 'text' column of both datasets
train_df['processed_text'] = train_df['text'].apply(preprocess_text)
test_df['processed_text'] = test_df['text'].apply(preprocess_text)

# Display preprocessed data samples
print("Preprocessed Train Data:")
print(train_df[['label-coarse', 'label-fine', 'processed_text']].head())
print("\nPreprocessed Test Data:")
print(test_df[['label-coarse', 'label-fine', 'processed_text']].head())

Preprocessed Train Data:
   label-coarse  label-fine                           processed_text
0             0           0             serfdom develop leave russia
1             1           1     film featured character popeye doyle
2             0           0            find list celebrity real name
3             1           2  fowl grab spotlight chinese year monkey
4             2           3                            full form com

Preprocessed Test Data:
   label-coarse  label-fine             processed_text
0             4          40           far denver aspen
1             5          21  county modesto california
2             3          12                    galileo
3             0           7                       atom
4             4           8        hawaii become state



# Task 2
# Train a Doc2Vec model on the data set and train a classifier based on the document embeddings of the train data set and the coarse labels.

In [21]:
pip install gensim

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import gensim
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


# Prepare tagged documents for Doc2Vec
train_documents = [TaggedDocument(words=row.split(), tags=[str(i)]) for i, row in enumerate(train_df['processed_text'])]
# Train Doc2Vec model
model_d2v = Doc2Vec(vector_size=300, window=5, min_count=1, workers=4, epochs=20)
model_d2v.build_vocab(train_documents)
model_d2v.train(train_documents, total_examples=model_d2v.corpus_count, epochs=model_d2v.epochs)

# Generate document embeddings for training data
train_embeddings = [model_d2v.infer_vector(doc.words) for doc in train_documents]

# Train a classifier (Logistic Regression) on the embeddings
X_train = train_embeddings
y_train = train_df['label-coarse']

# Split the data for training and validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


clf_rf = RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42)
clf_rf.fit(X_train_split, y_train_split)

y_val_pred_rf = clf_rf.predict(X_val_split)
print(classification_report(y_val_split, y_val_pred_rf))



/Users/oayanwale/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


              precision    recall  f1-score   support

           0       0.28      0.44      0.34       227
           1       0.30      0.34      0.32       246
           2       0.00      0.00      0.00        23
           3       0.30      0.38      0.34       240
           4       0.48      0.16      0.24       189
           5       0.48      0.24      0.32       166

    accuracy                           0.32      1091
   macro avg       0.31      0.26      0.26      1091
weighted avg       0.35      0.32      0.31      1091



/Users/oayanwale/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/oayanwale/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/oayanwale/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
import gensim
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


# Prepare tagged documents for Doc2Vec
train_documents = [TaggedDocument(words=row.split(), tags=[str(i)]) for i, row in enumerate(train_df['processed_text'])]
# Train Doc2Vec model
model_d2v = Doc2Vec(vector_size=300, window=5, min_count=1, workers=4, epochs=20)
model_d2v.build_vocab(train_documents)
model_d2v.train(train_documents, total_examples=model_d2v.corpus_count, epochs=model_d2v.epochs)

# Generate document embeddings for training data
train_embeddings = [model_d2v.infer_vector(doc.words) for doc in train_documents]

# Train a classifier (Logistic Regression) on the embeddings
X_train = train_embeddings
y_train = train_df['label-coarse']

# Split the data for training and validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


clf = LogisticRegression(max_iter=1000, class_weight='balanced')
clf.fit(X_train_split, y_train_split)

# Evaluate the model on the validation set
y_val_pred = clf.predict(X_val_split)
print(classification_report(y_val_split, y_val_pred))


              precision    recall  f1-score   support

           0       0.34      0.07      0.11       227
           1       0.29      0.09      0.14       246
           2       0.04      0.78      0.08        23
           3       0.30      0.48      0.37       240
           4       0.36      0.13      0.19       189
           5       0.16      0.10      0.13       166

    accuracy                           0.19      1091
   macro avg       0.25      0.28      0.17      1091
weighted avg       0.29      0.19      0.19      1091



Random Forest performed better overall in terms of accuracy, F1-score, and recall for most classes.
Logistic Regression struggles with imbalance—while it helped recover label 2, it misclassified many others.
Precision vs. Recall Tradeoff:
Logistic Regression: High recall for 2 but very low precision.
Random Forest: Better balance but still struggles with low-represented classes.


Further Improvements:
Use More Sophisticated Sampling Techniques: SMOTE (Synthetic Minority Over-sampling Technique)
Implement an ensemble of classifiers (e.g., combining Random Forest, Logistic Regression, etc.) to improve the overall performance.

Advanced BERT Adaptation (Task 3):

Proceeding to Task 3 (training a Lora adapter on top of BERT) could bring improvements, as it leverages the pre-trained knowledge of BERT while adapting to your specific task. BERT has been shown to outperform many traditional models for NLP tasks.

Hyperparameter Tuning:

Continue to experiment with hyperparameters for both the model (Doc2Vec, Random Forest) and the classifier (e.g., max_depth, min_samples_split in Random Forest).

# Task 3
# Train a lora adapter on top of BERT base uncased as a supervised model using the coarse labels of the training data set.

# Step 1: 
Install Dependencies

 pip install torch transformers peft datasets accelerate scikit-learn


In [8]:
#Step 2: Load Dataset

import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

# Ensure the label column is present
train_dataset = Dataset.from_pandas(train_df[['processed_text', 'label-coarse']])
test_dataset = Dataset.from_pandas(test_df[['processed_text', 'label-coarse']])

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["processed_text"], 
        padding="max_length", 
        truncation=True, 
        max_length=128
    )
    tokenized["labels"] = examples["label-coarse"]  # Add labels
    return tokenized

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Convert dataset to PyTorch tensors
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/5452 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
#step 3: Load BERT with LoRA Adapter
# We'll use Hugging Face PEFT to fine-tune LoRA on top of BERT.

from transformers import AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model

# Load BERT model for classification
num_labels = len(set(train_df["label-coarse"]))  # Number of classes
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Apply LoRA adapter
lora_config = LoraConfig(
    r=8,  # Rank of the LoRA matrices
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=["query", "value"]  # Apply LoRA to attention layers
)

model = get_peft_model(model, lora_config)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModel(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768

In [10]:
# Step 4: Define Training Arguments
# We'll set up the Trainer from Hugging Face to fine-tune the model.

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./lora_bert_coarse",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,  
    weight_decay=0.01,
    push_to_hub=False
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=None  # Optional, can add accuracy function
)

# Train model
trainer.train()


/Users/oayanwale/Library/Python/3.9/lib/python/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/cf/grsl9dbx6z9ckwfcv128r6hh0000gq/T/ipykernel_57833/1463592342.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/3410 [00:00<?, ?it/s]

{'loss': 1.7671, 'grad_norm': 5.3084940910339355, 'learning_rate': 4.266862170087977e-05, 'epoch': 0.73}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_runtime': 5.2347, 'eval_samples_per_second': 95.517, 'eval_steps_per_second': 12.035, 'epoch': 1.0}
{'loss': 1.6229, 'grad_norm': 1.4689269065856934, 'learning_rate': 3.533724340175953e-05, 'epoch': 1.47}


...

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_runtime': 5.1185, 'eval_samples_per_second': 97.686, 'eval_steps_per_second': 12.308, 'epoch': 4.0}
{'loss': 1.467, 'grad_norm': 2.895555257797241, 'learning_rate': 6.011730205278593e-06, 'epoch': 4.4}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_runtime': 5.1372, 'eval_samples_per_second': 97.33, 'eval_steps_per_second': 12.264, 'epoch': 5.0}
{'train_runtime': 733.71, 'train_samples_per_second': 37.154, 'train_steps_per_second': 4.648, 'train_loss': 1.5562311166891953, 'epoch': 5.0}


TrainOutput(global_step=3410, training_loss=1.5562311166891953, metrics={'train_runtime': 733.71, 'train_samples_per_second': 37.154, 'train_steps_per_second': 4.648, 'total_flos': 1799340423690240.0, 'train_loss': 1.5562311166891953, 'epoch': 5.0})

In [ ]:
# Check Model Performance on Evaluation Data
eval_results = trainer.evaluate()
print(eval_results)  # Check accuracy, precision, recall, or F1-score


# Task 4
# Fine tune a BERT base uncased model as a supervised model using the coarse labels of the training data set.


In [13]:
# Step 1: Load Dataset
# This step has already been done in Task 3, 
# so we can use the same datasets (train_dataset and test_dataset) for fine-tuning.


#Step 2: Load Pre-trained BERT Model
from transformers import AutoModelForSequenceClassification

# Load BERT base uncased model
num_labels = len(set(train_df["label-coarse"]))  # Number of unique labels in the training dataset
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# Step 3: Set up Training Arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert_coarse",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,  
    weight_decay=0.01,
    push_to_hub=False
)



/Users/oayanwale/Library/Python/3.9/lib/python/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
# Step 4: Define Trainer and Fine-tune the Model

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=None  # Optionally, you can define a function to compute accuracy or other metrics
)

# Train the model
trainer.train()


/var/folders/cf/grsl9dbx6z9ckwfcv128r6hh0000gq/T/ipykernel_57833/2831780326.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/3410 [00:00<?, ?it/s]

{'loss': 0.9237, 'grad_norm': 35.318214416503906, 'learning_rate': 4.266862170087977e-05, 'epoch': 0.73}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.4246484935283661, 'eval_runtime': 4.8276, 'eval_samples_per_second': 103.572, 'eval_steps_per_second': 13.05, 'epoch': 1.0}
{'loss': 0.5005, 'grad_norm': 16.90502166748047, 'learning_rate': 3.533724340175953e-05, 'epoch': 1.47}


...

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.5999035835266113, 'eval_runtime': 4.8734, 'eval_samples_per_second': 102.598, 'eval_steps_per_second': 12.927, 'epoch': 4.0}
{'loss': 0.0646, 'grad_norm': 0.042722318321466446, 'learning_rate': 6.011730205278593e-06, 'epoch': 4.4}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.5849773287773132, 'eval_runtime': 4.8308, 'eval_samples_per_second': 103.502, 'eval_steps_per_second': 13.041, 'epoch': 5.0}
{'train_runtime': 1031.6052, 'train_samples_per_second': 26.425, 'train_steps_per_second': 3.306, 'train_loss': 0.32397376113623116, 'epoch': 5.0}


TrainOutput(global_step=3410, training_loss=0.32397376113623116, metrics={'train_runtime': 1031.6052, 'train_samples_per_second': 26.425, 'train_steps_per_second': 3.306, 'total_flos': 1793166240430080.0, 'train_loss': 0.32397376113623116, 'epoch': 5.0})

In [16]:
# Step 5: Save the Fine-Tuned Model

model.save_pretrained("./bert_coarse_finetuned")
tokenizer.save_pretrained("./bert_coarse_finetuned")


('./bert_coarse_finetuned/tokenizer_config.json',
 './bert_coarse_finetuned/special_tokens_map.json',
 './bert_coarse_finetuned/vocab.txt',
 './bert_coarse_finetuned/added_tokens.json')

# Task 5
# Evaluate your three models on the test data set using a macro f1 score and compare their performances as well as the time it took to train them. Which model would you chose in which situation?


In [20]:
from transformers import AutoModelForSequenceClassification

# Load the LoRA model
lora_model = AutoModelForSequenceClassification.from_pretrained("/Users/oayanwale/Downloads/NLP_Exercise_24_25/Data/lora_model")


OSError: Incorrect path_or_model_id: '/Users/oayanwale/Downloads/NLP_Exercise_24_25/Data/lora_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [18]:
# Assuming the three models are:
# lora_model: BERT with LoRA
# finetuned_model: Fine-tuned BERT
# clf_rf: Doc2Vec + Random Forest

from sklearn.metrics import f1_score
import time


# Function to evaluate BERT models (LoRA and Fine-tuned)
def evaluate_model(model, test_dataset, is_bert=True):
    model.eval()
    all_preds = []
    all_labels = []
    
    # For BERT models
    if is_bert:
        for batch in test_dataset:
            inputs = {key: batch[key].to(device) for key in ['input_ids', 'attention_mask']}
            labels = batch['labels'].to(device)
            
            with torch.no_grad():
                outputs = model(**inputs)
            
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    else:
        # For Doc2Vec + RandomForest
        X_test = [model_d2v.infer_vector(doc.split()) for doc in test_df['processed_text']]  # Infer vectors for test data
        y_test = test_df['label-coarse']
        all_preds = clf_rf.predict(X_test)
        all_labels = y_test

    # Calculate Macro F1 score
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    return macro_f1

# Evaluate BERT with LoRA
start_time = time.time()
macro_f1_lora = evaluate_model(lora_model, test_dataset)
lora_time = time.time() - start_time

# Evaluate Fine-Tuned BERT
start_time = time.time()
macro_f1_bert = evaluate_model(finetuned_model, test_dataset)
bert_time = time.time() - start_time

# Evaluate Doc2Vec + Random Forest
start_time = time.time()
macro_f1_rf = evaluate_model(clf_rf, test_dataset, is_bert=False)
rf_time = time.time() - start_time

# Print Results
print(f"BERT with LoRA Macro F1: {macro_f1_lora:.4f}, Time Taken: {lora_time:.2f}s")
print(f"Fine-Tuned BERT Macro F1: {macro_f1_bert:.4f}, Time Taken: {bert_time:.2f}s")
print(f"Doc2Vec + Random Forest Macro F1: {macro_f1_rf:.4f}, Time Taken: {rf_time:.2f}s")


OSError: Incorrect path_or_model_id: './path_to_saved_lora_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [17]:
from sklearn.metrics import f1_score

def evaluate_model(trainer, eval_dataset):
    predictions = trainer.predict(eval_dataset)
    preds = predictions.predictions.argmax(-1)  # Get predicted classes
    labels = predictions.label_ids
    
    # Calculate macro F1 score
    f1_macro = f1_score(labels, preds, average='macro')
    
    return f1_macro

# Evaluate both models
f1_lora = evaluate_model(trainer, eval_dataset)
f1_finetuned = evaluate_model(trainer_finetuned, eval_dataset)

print(f"Macro F1 Score (LoRA): {f1_lora}")
print(f"Macro F1 Score (Fine-tuned BERT): {f1_finetuned}")

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

Macro F1 Score (LoRA): 0.683577031698368
Macro F1 Score (Fine-tuned BERT): 0.5147557544717821


# Task 6
# Repeat tasks 2-5 with the fine grained labels instead. Has anything changed?


In [18]:
train_labels_fine = train_df['label-fine'].tolist()
test_labels_fine = test_df['label-fine'].tolist()

train_dataset_fine = CustomDataset(train_encodings_lora, train_labels_fine)
eval_dataset_fine = CustomDataset(test_encodings_lora, test_labels_fine)

In [19]:
# Prepare tokenized inputs for training dataset with fine-grained labels
train_encodings_fine = tokenizer(list(train_df['processed_text']), truncation=True, padding=True)
train_labels_fine = train_df['label-fine'].tolist()


In [33]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)


In [21]:
# Create training dataset with fine-grained labels
train_dataset_fine = CustomDataset(train_encodings_fine, train_labels_fine)

# Prepare evaluation dataset using your test data with fine-grained labels
test_encodings_fine = tokenizer(list(test_df['processed_text']), truncation=True, padding=True)
test_labels_fine = test_df['label-fine'].tolist()

eval_dataset_fine = CustomDataset(test_encodings_fine, test_labels_fine)

# Initialize LoRA adapter configuration using LoraConfig if needed here...
lora_adapter_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
)


In [23]:
import torch
from transformers import BertModel

class CustomBertWithLoRA(torch.nn.Module):
    def __init__(self, num_classes):
        super(CustomBertWithLoRA, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # Use pooler output for classification
        logits = self.classifier(pooled_output)

        if labels is not None:
            # Compute loss using CrossEntropyLoss
            loss_fn = torch.nn.CrossEntropyLoss()
            loss = loss_fn(logits.view(-1, self.classifier.out_features), labels.view(-1))
            return {"loss": loss, "logits": logits}

        return {"logits": logits}

# Initialize your custom model with fine-grained labels
num_classes_fine_grained = len(train_df['label-fine'].unique())  # Adjust based on your dataset
model_lora_adapter_for_finetuning = CustomBertWithLoRA(num_classes_fine_grained)

# Wrap it with LoRA if needed
model_lora_adapter_with_peft = get_peft_model(model_lora_adapter_for_finetuning, lora_adapter_config)

/Users/oayanwale/Library/Python/3.9/lib/python/site-packages/peft/mapping.py:185: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'bert-base-uncased' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


In [46]:
import torch
from transformers import BertModel

class CustomBertClassifier(torch.nn.Module):
    def __init__(self, num_classes):
        super(CustomBertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fn = torch.nn.CrossEntropyLoss()
            loss = loss_fn(logits.view(-1, self.classifier.out_features), labels.view(-1))
            return {"loss": loss}

        return {"logits": logits}

In [47]:
# Assuming train_df and test_df are already loaded and preprocessed.
train_labels_fine = train_df['label-fine'].tolist()  # Use fine-grained labels here
test_labels_fine = test_df['label-fine'].tolist()

# Tokenize inputs for fine-grained datasets
train_encodings_fine = tokenizer(list(train_df['processed_text']), truncation=True, padding=True)
test_encodings_fine = tokenizer(list(test_df['processed_text']), truncation=True, padding=True)

# Create datasets for fine-grained classification
train_dataset_fine = CustomDataset(train_encodings_fine, train_labels_fine)
eval_dataset_fine = CustomDataset(test_encodings_fine, test_labels_fine)

In [24]:

# Initialize LoRA model with BERT base uncased.
model_lora_adapter_for_finetuning = get_peft_model(BertModel.from_pretrained("bert-base-uncased"), lora_adapter_config)

# Set up training arguments for LoRA model training
training_args_lora_fine_grained = TrainingArguments(
    output_dir='/Users/oayanwale/Documents/NLP_LoRA_FineGrained_Results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    evaluation_strategy="epoch",
)

# Train LoRA model with fine-grained labels using Trainer API
trainer_with_lora_for_fine_labels = Trainer(
    model=model_lora_adapter_with_peft,
    args=training_args_lora_fine_grained,
    train_dataset=train_dataset_fine,
    eval_dataset=eval_dataset_fine  # Pass eval dataset here for evaluation during training 
)

trainer_with_lora_for_fine_labels.train()

  0%|          | 0/1023 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_runtime': 1.5204, 'eval_samples_per_second': 328.85, 'eval_steps_per_second': 41.435, 'epoch': 1.0}
{'loss': 3.5721, 'grad_norm': 1.8073559999465942, 'learning_rate': 2.5562072336265887e-05, 'epoch': 1.47}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_runtime': 1.3495, 'eval_samples_per_second': 370.499, 'eval_steps_per_second': 46.683, 'epoch': 2.0}
{'loss': 3.3703, 'grad_norm': 2.6283085346221924, 'learning_rate': 1.1241446725317694e-06, 'epoch': 2.93}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_runtime': 1.7156, 'eval_samples_per_second': 291.446, 'eval_steps_per_second': 36.722, 'epoch': 3.0}
{'train_runtime': 112.3594, 'train_samples_per_second': 145.569, 'train_steps_per_second': 9.105, 'train_loss': 3.469629729598149, 'epoch': 3.0}


TrainOutput(global_step=1023, training_loss=3.469629729598149, metrics={'train_runtime': 112.3594, 'train_samples_per_second': 145.569, 'train_steps_per_second': 9.105, 'total_flos': 0.0, 'train_loss': 3.469629729598149, 'epoch': 3.0})

In [50]:
# Create training dataset with fine-grained labels 
train_dataset_fine_grained = CustomDataset(train_encodings_fine, train_labels_fine)

# Prepare your eval dataset using your test data (fine-grained this time)
test_encodings_fine = tokenizer(list(test_df['processed_text']), truncation=True, padding=True)
test_labels_fine = test_df['label-fine'].tolist() 

eval_dataset_fine_grained = CustomDataset(test_encodings_fine, test_labels_fine)


# Set up training arguments for LoRA model with a writable output directory 
training_args_lora_fine_grained = TrainingArguments(
    output_dir='/Users/oayanwale/Documents/NLP_Finetuned_FG_LoRA',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    evaluation_strategy="epoch",
)

# Initialize Trainer with eval_dataset included now 
trainer_lora_fine_grained = Trainer(
    model=model_lora_adapter,
    args=training_args_lora_fine_grained,
    train_dataset=train_dataset_fine_grained,
    eval_dataset=eval_dataset_fine_grained  
)

# Start training process 
trainer_lora_fine_grained.train()


  0%|          | 0/1023 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.08316250890493393, 'eval_runtime': 1.4562, 'eval_samples_per_second': 343.348, 'eval_steps_per_second': 43.262, 'epoch': 1.0}
{'loss': 0.1686, 'grad_norm': 16.576438903808594, 'learning_rate': 2.5562072336265887e-05, 'epoch': 1.47}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.06176266819238663, 'eval_runtime': 1.3624, 'eval_samples_per_second': 366.992, 'eval_steps_per_second': 46.241, 'epoch': 2.0}
{'loss': 0.0367, 'grad_norm': 20.284021377563477, 'learning_rate': 1.1241446725317694e-06, 'epoch': 2.93}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.06864174455404282, 'eval_runtime': 1.4333, 'eval_samples_per_second': 348.844, 'eval_steps_per_second': 43.954, 'epoch': 3.0}
{'train_runtime': 183.5302, 'train_samples_per_second': 89.119, 'train_steps_per_second': 5.574, 'train_loss': 0.10114389523965756, 'epoch': 3.0}


TrainOutput(global_step=1023, training_loss=0.10114389523965756, metrics={'train_runtime': 183.5302, 'train_samples_per_second': 89.119, 'train_steps_per_second': 5.574, 'total_flos': 0.0, 'train_loss': 0.10114389523965756, 'epoch': 3.0})

In [51]:
### Step 4: Fine-Tune Standard BERT Model with Fine-Grained Labels

class CustomBertClassifier(torch.nn.Module):
    def __init__(self, num_classes):
        super(CustomBertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fn = torch.nn.CrossEntropyLoss()
            loss = loss_fn(logits.view(-1), labels.view(-1))
            return {"loss": loss}

        return {"logits": logits}


In [52]:
### Step 5: Evaluate Both Models Using Macro F1 Score

def evaluate_model(trainer_instance_name_here..., eval_dataset_name_here...):
   predictions_output_evaluated_against_eval_datasets...
   preds_evaluated_against_predicted_class_and_returned_logits...
   f1_macro_score_result_calculated_using_sklearn_metrics_module...

evaluate_model(trainer_lora_fine_grained.eval_datasets_evaluated_against_eval_data_set...)

evaluate_model(trainer_standard_bert_with_eval_on_test_data_set.eval_datasets_evaluated_against_eval_data_set...)

print(f"Macro F1 Score (LoRA - Fine Grained): {f1_score_result}")
print(f"Macro F1 Score (Fine-tuned BERT - Fine Grained): {f1_score_result}")


SyntaxError: invalid syntax (4205328198.py, line 3)